# Elasticity Tool (Simple Version)

Interactive tool for calculating price elasticity using real market data from Athens soda sales.

**Instructions:**
1. Run the cell below
2. Wait for packages to install (first time only)
3. Use the interactive widgets to calculate elasticities

In [ ]:
# Install required packages for JupyterLite
import micropip
await micropip.install(['pandas', 'numpy', 'matplotlib', 'scikit-learn', 'ipywidgets'])

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from IPython.display import display, clear_output
import ipywidgets as widgets

# Load and prepare data
df = pd.read_csv('soda.csv')
df['date'] = pd.to_datetime(df['date'])

df_mean_Q = df.groupby(['brand','container','city','date'])['quantity'].mean().reset_index()
df_mean_P = df.groupby(['brand','container','city','date'])['price'].mean().reset_index()
df_city = pd.merge(df_mean_Q, df_mean_P)
df_city.rename(columns={'quantity': 'mean_q', 'price': 'mean_p'}, inplace=True)
df_city = df_city.pivot(index='date', columns=['city','brand','container'], values=['mean_q','mean_p'])

brands = ['adult-cola', 'gazoza', 'kinder-cola', 'lemon-boost', 'orange-power']
containers = ['plastic', 'can', 'glass']

# Analysis functions
def calculate_own_price_elasticity(brand, container, price_point):
    try:
        Qx = df_city['mean_q']['Athens'][brand][container].values.reshape(-1,1)
        Px = df_city['mean_p']['Athens'][brand][container].values.reshape(-1,1)
        
        # Plot relationship
        plt.figure(figsize=(8, 6))
        reg = LinearRegression().fit(Px, Qx)
        Px_plot = np.linspace(Px.min(), Px.max(), 100).reshape(-1,1)
        Qx_pred = reg.predict(Px_plot)
        
        plt.scatter(Qx, Px, alpha=0.7, label='Data')
        plt.plot(Qx_pred, Px_plot, color='red', label='Regression Line')
        plt.xlabel(f'Quantity ({brand} {container})')
        plt.ylabel(f'Price ({brand} {container})')
        plt.title(f'Price vs Quantity: {brand} {container}')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
        
        # Calculate elasticity
        Q_hat = reg.predict(np.array(price_point).reshape(-1, 1))
        elasticity = reg.coef_[0][0] * (price_point / Q_hat[0][0])
        
        print(f'Own-Price Elasticity: {elasticity:.3f}')
        if elasticity < -1:
            print('Demand is ELASTIC (responsive to price changes)')
        elif elasticity > -1 and elasticity < 0:
            print('Demand is INELASTIC (less responsive to price changes)')
        else:
            print('Unusual elasticity value - check data')
            
    except Exception as e:
        print(f'Error calculating elasticity: {str(e)}')

def calculate_cross_price_elasticity(brand1, container1, brand2, container2, price_point):
    try:
        Qx = df_city['mean_q']['Athens'][brand1][container1].values.reshape(-1,1)
        Px1 = df_city['mean_p']['Athens'][brand1][container1].values.reshape(-1,1)
        Px2 = df_city['mean_p']['Athens'][brand2][container2].values.reshape(-1,1)
        Px = np.concatenate((Px1, Px2), axis=1)
        
        reg = LinearRegression().fit(Px, Qx)
        mPx1 = np.mean(Px1)
        P = np.array([[mPx1, price_point]])
        Q_hat = reg.predict(P)
        cross_elasticity = reg.coef_[0][1] * (price_point / Q_hat[0][0])
        
        print(f'Cross-Price Elasticity ({brand2} price effect on {brand1} demand): {cross_elasticity:.3f}')
        if cross_elasticity > 0:
            print('Products are SUBSTITUTES (price increase of one increases demand for other)')
        elif cross_elasticity < 0:
            print('Products are COMPLEMENTS (price increase of one decreases demand for other)')
        else:
            print('Products are INDEPENDENT (no cross-price effect)')
            
    except Exception as e:
        print(f'Error calculating cross-price elasticity: {str(e)}')

# Interactive widgets
brand_dropdown = widgets.Dropdown(options=brands, value='adult-cola', description='Brand:')
container_dropdown = widgets.Dropdown(options=containers, value='plastic', description='Container:')
price_slider = widgets.FloatSlider(min=0.5, max=5.0, step=0.1, value=2.0, description='Price Point:')

brand2_dropdown = widgets.Dropdown(options=brands, value='gazoza', description='Brand 2:')
container2_dropdown = widgets.Dropdown(options=containers, value='plastic', description='Container 2:')
price2_slider = widgets.FloatSlider(min=0.5, max=5.0, step=0.1, value=2.0, description='Price Point 2:')

own_elasticity_button = widgets.Button(description='Calculate Own-Price Elasticity', button_style='primary')
cross_elasticity_button = widgets.Button(description='Calculate Cross-Price Elasticity', button_style='info')
output = widgets.Output()

def on_own_elasticity_click(b):
    with output:
        clear_output()
        print(f"Analyzing {brand_dropdown.value} {container_dropdown.value} at price ${price_slider.value}")
        print("=" * 60)
        calculate_own_price_elasticity(brand_dropdown.value, container_dropdown.value, price_slider.value)

def on_cross_elasticity_click(b):
    with output:
        clear_output()
        print(f"Cross-elasticity: {brand2_dropdown.value} price effect on {brand_dropdown.value} demand")
        print("=" * 60)
        calculate_cross_price_elasticity(
            brand_dropdown.value, container_dropdown.value,
            brand2_dropdown.value, container2_dropdown.value,
            price2_slider.value
        )

own_elasticity_button.on_click(on_own_elasticity_click)
cross_elasticity_button.on_click(on_cross_elasticity_click)

# Layout
own_price_section = widgets.VBox([
    widgets.HTML('<h3>Own-Price Elasticity Analysis</h3>'),
    brand_dropdown,
    container_dropdown,
    price_slider,
    own_elasticity_button
])

cross_price_section = widgets.VBox([
    widgets.HTML('<h3>Cross-Price Elasticity Analysis</h3>'),
    widgets.HTML('<b>Product 1 (demand):</b>'),
    brand_dropdown,
    container_dropdown,
    widgets.HTML('<b>Product 2 (price change):</b>'),
    brand2_dropdown,
    container2_dropdown,
    price2_slider,
    cross_elasticity_button
])

main_interface = widgets.HBox([
    own_price_section,
    widgets.HTML('<div style="width: 50px;"></div>'),
    cross_price_section
])

# Display interface and data preview
print("Elasticity Analysis Tool - Athens Soda Market Data")
print("=" * 60)
print("Sample of the data:")
display(df[['date', 'brand', 'container', 'price', 'quantity']].head())
print("\nAvailable brands:", ", ".join(brands))
print("Available containers:", ", ".join(containers))
print("\nUse the controls below to analyze price elasticities:")

display(main_interface)
display(output);